In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4MB 69kB/s 
     |████████████████████████████████| 204kB 14.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=2a65de1c592e3c8160819e2565e3ea787a63883602294177a04cfd2246fab0b5
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.5 MB of archives.
After this 

In [ ]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1f_9EbnywCj35EBUA32sueigxBjBJwALr'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('War and Peace by Leo Tolstoy (ru).txt')

In [ ]:
import pandas as pd

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().set("spark.ui.port", "4050")

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

*`ngrok может отработать не с первого раза, повторите при необходимости.`*

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-05-31 15:41:22--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.193.189.47, 34.239.208.97, 3.220.114.126, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.193.189.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  4.76MB/s    in 2.8s    

2021-05-31 15:41:25 (4.76 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://21ec7b7746cc.ngrok.io


Подсчитай кол-во слов в документе "War and Peace by Leo Tolstoy (ru).txt"

In [ ]:
# код
tokens = sc.textFile("War and Peace by Leo Tolstoy (ru).txt").flatMap(lambda line: line.split(" "))

In [ ]:
word_counts = tokens.map(lambda word: ('total', 1)).reduceByKey(lambda w1, w2: w1 + w2)

In [ ]:
collect_words = word_counts.collect()
collect_words[0][1]

493897

In [ ]:
id='13yfAoONwq4rS5XrTv3IrcqcFcdgfvK9V'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1VE_9x0LQvOJpHXbXp_RMPl3Q4wRUuOok'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

In [ ]:
training = spark.read.format("libsvm") \
  .option("numFeatures", "784") \
  .load("./mnist-digits-train.txt")
test = spark.read.format("libsvm") \
  .option("numFeatures", "784") \
  .load("./mnist-digits-test.txt")

Необходимо обучить модель используя Spark MLlib (модель на ваш выбор, например Decision Tree) и получить accuracy.
Подробнее тут: https://spark.apache.org/docs/latest/ml-classification-regression.html

Разбиваем тренировочную выборку на train и test

In [ ]:
(trainingData, testData) = training.randomSplit([0.7, 0.3])

задаем алгоритм. В данном случае используем LogisticRegression

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

stages = []
lr = LogisticRegression(labelCol="label", featuresCol="features", regParam=0.001)
stages += [lr]

Создаем pipeline

In [ ]:
pipeline = Pipeline(stages=stages)

Задаем сетку параметров

In [ ]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam, [10, 1.0, 0.1, 0.01, 0.001]).build()

Проводим кросс-валидацию

In [ ]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=5)
cvModel = crossval.fit(trainingData)

Смотрим лучшие параметры

In [ ]:
cvModel.bestModel.stages[-1]._java_obj.getRegParam()

0.001

Предиктим

In [ ]:
prediction = cvModel.transform(testData)

Считаем метрику

In [ ]:
labelAndPred = prediction.select('label', 'prediction')
metrics_multi = MulticlassMetrics(labelAndPred.rdd.map(tuple))
acc = metrics_multi.accuracy
acc

0.9224569205113953

Теперь обучим модель на полном наборе данных и сделаем предсказание

In [ ]:
model = pipeline.fit(training)

predictions = model.transform(test)

Выберем столбец prediction

In [ ]:
y_pred = predictions.select('prediction')
y_pred.show()

+----------+
|prediction|
+----------+
|       7.0|
|       2.0|
|       1.0|
|       0.0|
|       4.0|
|       1.0|
|       4.0|
|       9.0|
|       6.0|
|       9.0|
|       0.0|
|       6.0|
|       9.0|
|       0.0|
|       1.0|
|       5.0|
|       9.0|
|       7.0|
|       3.0|
|       4.0|
+----------+
only showing top 20 rows



Тут лежат данные для решения задач на sql. Необходимо написать код который формирует из данных датафреймы и используя sqlContext написать sql код с решением. Приветствуется наличие нескольких вариантов решения задачи.

In [ ]:
id='1kUIrskM0zNH8u71G9M1BkHjRQYxvgAvh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('data.zip')
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/regions.csv        
  inflating: data/departments.csv    
  inflating: data/jobs.csv           
  inflating: data/locations.csv      
  inflating: data/country.csv        
  inflating: data/employees.csv      
  inflating: data/job_history.csv    


In [ ]:
regions = spark.read.csv('data/regions.csv', header=True, sep='\t')
regions.createOrReplaceTempView("regions")

In [ ]:
regions.show()

+---------+--------------------+
|REGION_ID|         REGION_NAME|
+---------+--------------------+
|        1|              Europe|
|        2|            Americas|
|        3|                Asia|
|        4|Middle East and A...|
+---------+--------------------+



In [ ]:
dep = spark.read.csv('data/departments.csv', header=True, sep='\t')
dep.createOrReplaceTempView("department")

In [ ]:
dep.show()

+-------------+--------------------+----------+-----------+
|DEPARTMENT_ID|     DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+--------------------+----------+-----------+
|           10|      Administration|       200|       1700|
|           20|           Marketing|       201|       1800|
|           30|          Purchasing|       114|       1700|
|           40|     Human Resources|       203|       2400|
|           50|            Shipping|       121|       1500|
|           60|                  IT|       103|       1400|
|           70|    Public Relations|       204|       2700|
|           80|               Sales|       145|       2500|
|           90|           Executive|       100|       1700|
|          100|             Finance|       108|       1700|
|          110|          Accounting|       205|       1700|
|          120|            Treasury|      null|       1700|
|          130|       Corporate Tax|      null|       1700|
|          140|  Control And Credit|    

In [ ]:
jobs = spark.read.csv('data/jobs.csv', header=True, sep='\t')
jobs.createOrReplaceTempView("jobs")

In [ ]:
jobs.show()

+----------+--------------------+----------+----------+
|    JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|
+----------+--------------------+----------+----------+
|   AD_PRES|           President|     20080|     40000|
|     AD_VP|Administration Vi...|     15000|     30000|
|   AD_ASST|Administration As...|      3000|      6000|
|    FI_MGR|     Finance Manager|      8200|     16000|
|FI_ACCOUNT|          Accountant|      4200|      9000|
|    AC_MGR|  Accounting Manager|      8200|     16000|
|AC_ACCOUNT|   Public Accountant|      4200|      9000|
|    SA_MAN|       Sales Manager|     10000|     20080|
|    SA_REP|Sales Representative|      6000|     12008|
|    PU_MAN|  Purchasing Manager|      8000|     15000|
|  PU_CLERK|    Purchasing Clerk|      2500|      5500|
|    ST_MAN|       Stock Manager|      5500|      8500|
|  ST_CLERK|         Stock Clerk|      2008|      5000|
|  SH_CLERK|      Shipping Clerk|      2500|      5500|
|   IT_PROG|          Programmer|      4000|    

In [ ]:
locations = spark.read.csv('data/locations.csv', header=True, sep='\t')
locations.createOrReplaceTempView("locations")

In [ ]:
locations.show()

+-----------+--------------------+-----------+-------------------+----------------+----------+
|LOCATION_ID|      STREET_ADDRESS|POSTAL_CODE|               CITY|  STATE_PROVINCE|COUNTRY_ID|
+-----------+--------------------+-----------+-------------------+----------------+----------+
|       1000|1297 Via Cola di Rie|      00989|               Roma|            null|        IT|
|       1100|93091 Calle della...|      10934|             Venice|            null|        IT|
|       1200|    2017 Shinjuku-ku|       1689|              Tokyo|Tokyo Prefecture|        JP|
|       1300|     9450 Kamiya-cho|       6823|          Hiroshima|            null|        JP|
|       1400| 2014 Jabberwocky Rd|      26192|          Southlake|           Texas|        US|
|       1500| 2011 Interiors Blvd|      99236|South San Francisco|      California|        US|
|       1600|      2007 Zagora St|      50090|    South Brunswick|      New Jersey|        US|
|       1700|     2004 Charade Rd|      98199|    

In [ ]:
country = spark.read.csv('data/country.csv', header=True, sep='\t')
country.createOrReplaceTempView("country")

In [ ]:
country.show()

+----------+------------+---------+
|COUNTRY_ID|COUNTRY_NAME|REGION_ID|
+----------+------------+---------+
|        AR|   Argentina|        2|
|        AU|   Australia|        3|
|        BE|     Belgium|        1|
|        BR|      Brazil|        2|
|        CA|      Canada|        2|
|        CH| Switzerland|        1|
|        CN|       China|        3|
|        DE|     Germany|        1|
|        DK|     Denmark|        1|
|        EG|       Egypt|        4|
|        FR|      France|        1|
|        IL|      Israel|        4|
|        IN|       India|        3|
|        IT|       Italy|        1|
|        JP|       Japan|        3|
|        KW|      Kuwait|        4|
|        ML|    Malaysia|        3|
|        MX|      Mexico|        2|
|        NG|     Nigeria|        4|
|        NL| Netherlands|        1|
+----------+------------+---------+
only showing top 20 rows



In [ ]:
emp = spark.read.csv('data/employees.csv', header=True, sep='\t')
emp.createOrReplaceTempView("employees")

In [ ]:
emp.show()

+-----------+-----------+----------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID| FIRST_NAME| LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+-----------+----------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        100|     Steven|      King|   SKING|515.123.4567| 17.06.03|   AD_PRES| 24000|          null|      null|           90|
|        101|      Neena|   Kochhar|NKOCHHAR|515.123.4568| 21.09.05|     AD_VP| 17000|          null|       100|           90|
|        102|        Lex|   De Haan| LDEHAAN|515.123.4569| 13.01.01|     AD_VP| 17000|          null|       100|           90|
|        103|  Alexander|    Hunold| AHUNOLD|590.423.4567| 03.01.06|   IT_PROG|  9000|          null|       102|           60|
|        104|      Bruce|     Ernst|  BERNST|590.423.4568| 21.05.07|   IT_PROG|  6000|          null|       103

In [ ]:
j_his = spark.read.csv('data/job_history.csv', header=True, sep='\t')
j_his.createOrReplaceTempView("job_history")

In [ ]:
j_his.show()

+-----------+----------+--------+----------+-------------+
|EMPLOYEE_ID|START_DATE|END_DATE|    JOB_ID|DEPARTMENT_ID|
+-----------+----------+--------+----------+-------------+
|        102|  13.01.01|24.07.06|   IT_PROG|           60|
|        101|  21.09.97|27.10.01|AC_ACCOUNT|          110|
|        101|  28.10.01|15.03.05|    AC_MGR|          110|
|        201|  17.02.04|19.12.07|    MK_REP|           20|
|        114|  24.03.06|31.12.07|  ST_CLERK|           50|
|        122|  01.01.07|31.12.07|  ST_CLERK|           50|
|        200|  17.09.95|17.06.01|   AD_ASST|           90|
|        176|  24.03.06|31.12.06|    SA_REP|           80|
|        176|  01.01.07|31.12.07|    SA_MAN|           80|
|        200|  01.07.02|31.12.06|AC_ACCOUNT|           90|
+-----------+----------+--------+----------+-------------+



Кто получает больше всего? Кто меньше всего?

Больше всего

In [ ]:
# код
sqlDF = spark.sql("""SELECT EMPLOYEE_ID, FIRST_NAME, LAST_NAME, SALARY 
                  FROM employees
                  WHERE SALARY IN (SELECT MAX(CAST(SALARY AS INT))
                                   FROM employees)
                  """)
sqlDF.show()

+-----------+----------+---------+------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|SALARY|
+-----------+----------+---------+------+
|        100|    Steven|     King| 24000|
+-----------+----------+---------+------+



Меньше всего

In [ ]:
# код
sqlDF = spark.sql("""SELECT EMPLOYEE_ID, FIRST_NAME, LAST_NAME, SALARY 
                  FROM employees
                  WHERE SALARY IN (SELECT MIN(CAST(SALARY AS INT))
                                   FROM employees)
                  """)
sqlDF.show()

+-----------+----------+---------+------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|SALARY|
+-----------+----------+---------+------+
|        132|        TJ|    Olson|  2100|
+-----------+----------+---------+------+



Выведете топ 5 по зарплате.

In [ ]:
# код
sqlDF = spark.sql("""SELECT EMPLOYEE_ID, FIRST_NAME, LAST_NAME, CAST(SALARY as INT) as SALARY
                     FROM employees
                     ORDER BY SALARY DESC
                     LIMIT 5
""")
sqlDF.show()

+-----------+----------+---------+------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|SALARY|
+-----------+----------+---------+------+
|        100|    Steven|     King| 24000|
|        101|     Neena|  Kochhar| 17000|
|        102|       Lex|  De Haan| 17000|
|        145|      John|  Russell| 14000|
|        146|     Karen| Partners| 13500|
+-----------+----------+---------+------+



Сколько всего регионов? Сколько работников в каждом регионе?

Сколько всего регионов

In [ ]:
# код
sqlDF = spark.sql("""SELECT COUNT(*) AS counter_regions
                     FROM (SELECT DISTINCT REGION_NAME
                           FROM regions)
""")
sqlDF.show()

+---------------+
|counter_regions|
+---------------+
|              4|
+---------------+



Сколько работников в каждом регионе

In [ ]:
sqlDF = spark.sql("""WITH cte AS(
                          SELECT dep.DEPARTMENT_ID, loc.COUNTRY_ID
                          FROM department AS dep INNER JOIN locations as loc
                          ON dep.LOCATION_ID = loc.LOCATION_ID),
                     cte2 AS(
                          SELECT country.REGION_ID, cte.DEPARTMENT_ID
                          FROM cte INNER JOIN country
                          ON country.COUNTRY_ID = cte.COUNTRY_ID)

                    SELECT regions.REGION_NAME, 
                    CASE WHEN counter_emp IS NULL
                    THEN 0
                    ELSE counter_emp
                    END counter_emp
                    FROM regions LEFT JOIN (
                                                SELECT cte2.REGION_ID, count(emp.EMPLOYEE_ID) as counter_emp
                                                FROM cte2 INNER JOIN employees AS emp
                                                ON cte2.DEPARTMENT_ID = emp.DEPARTMENT_ID
                                                GROUP BY cte2.REGION_ID) as t
                    ON regions.REGION_ID = t.REGION_ID
                    """)
sqlDF.show()

+--------------------+-----------+
|         REGION_NAME|counter_emp|
+--------------------+-----------+
|                Asia|          0|
|              Europe|         36|
|Middle East and A...|          0|
|            Americas|         70|
+--------------------+-----------+



Выведете всех работников из Китая.

In [ ]:
# код
sqlDF = spark.sql("""WITH cte AS(
                          SELECT dep.DEPARTMENT_ID, loc.COUNTRY_ID
                          FROM department AS dep INNER JOIN locations as loc
                          ON dep.LOCATION_ID = loc.LOCATION_ID)
                    SELECT emp.EMPLOYEE_ID, emp.FIRST_NAME, emp.LAST_NAME, cte.COUNTRY_ID
                    FROM cte INNER JOIN employees AS emp
                    ON cte.DEPARTMENT_ID = emp.DEPARTMENT_ID
                    WHERE cte.COUNTRY_ID = 'CN'
                    """)
sqlDF.show()

+-------------+----------+
|DEPARTMENT_ID|COUNTRY_ID|
+-------------+----------+
+-------------+----------+



Укажите самую высокооплачиваемою должность.

In [ ]:
# код
sqlDF = spark.sql("""SELECT JOB_TITLE, MAX_SALARY
                     FROM jobs
                     WHERE MAX_SALARY IN (SELECT MAX(CAST(MAX_SALARY AS INT))
                                          FROM jobs)""")
sqlDF.show()

+---------+----------+
|JOB_TITLE|MAX_SALARY|
+---------+----------+
|President|     40000|
+---------+----------+



Выведете всех работников связанных с ИТ. Выведете их менеджеров. 

In [ ]:
# код
sqlDF = spark.sql("""SELECT EMPLOYEE_ID, FIRST_NAME, LAST_NAME, MANAGER_ID
                     FROM employees
                     WHERE JOB_ID = 'IT_PROG'
""")
sqlDF.show()

+-----------+----------+---------+----------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|MANAGER_ID|
+-----------+----------+---------+----------+
|        103| Alexander|   Hunold|       102|
|        104|     Bruce|    Ernst|       103|
|        105|     David|   Austin|       103|
|        106|     Valli|Pataballa|       103|
|        107|     Diana|  Lorentz|       103|
+-----------+----------+---------+----------+



Выведете имя и фамилию работника, его текущую и предыдущую должности и сколько полных недель и дней прошло с момент изменения. 



In [ ]:
# код
sqlDF = spark.sql("""WITH cte AS(
                     SELECT jb.EMPLOYEE_ID, TO_DATE(CAST(UNIX_TIMESTAMP(emp.HIRE_DATE, 'dd.MM.yy') AS TIMESTAMP)) as START_DATE,
                     TO_DATE(CAST(UNIX_TIMESTAMP(jb.END_DATE, 'dd.MM.yy') AS TIMESTAMP)) as END_DATE,
                     jb.JOB_ID as PREV_JOB, emp.JOB_ID as NOW_JOB,
                     emp.FIRST_NAME, emp.LAST_NAME,
                     RANK() OVER(PARTITION BY jb.EMPLOYEE_ID ORDER BY END_DATE DESC) as RANK_JOB
                     FROM job_history as jb JOIN employees as emp
                     ON jb.EMPLOYEE_ID = emp.EMPLOYEE_ID
                     ORDER BY jb.EMPLOYEE_ID),
                     cte2 AS(
                         SELECT *
                         FROM cte
                         WHERE RANK_JOB = 1 and START_DATE > END_DATE
                     )

                     SELECT FIRST_NAME, LAST_NAME, PREV_JOB, NOW_JOB, DATEDIFF(START_DATE, END_DATE) as DIFF_DAYS,
                     CAST(DATEDIFF(START_DATE, END_DATE) / 30 AS INT) as DIFF_MONTHS FROM cte2
""")
#CAST(UNIX_TIMESTAMP(START_DATE, 'dd.MM.yy') AS TIMESTAMP)
sqlDF.show()

+----------+---------+----------+-------+---------+-----------+
|FIRST_NAME|LAST_NAME|  PREV_JOB|NOW_JOB|DIFF_DAYS|DIFF_MONTHS|
+----------+---------+----------+-------+---------+-----------+
|     Neena|  Kochhar|AC_ACCOUNT|  AD_VP|     1425|         47|
+----------+---------+----------+-------+---------+-----------+



Выведете уникальные телефонные номера

In [ ]:
# код
sqlDF = spark.sql("""SELECT DISTINCT PHONE_NUMBER
                     FROM employees
""")
sqlDF.show()

+------------------+
|      PHONE_NUMBER|
+------------------+
|011.44.1344.429018|
|      515.127.4566|
|      515.127.4564|
|011.44.1344.429278|
|      515.123.4569|
|      650.124.1434|
|      650.123.2234|
|011.44.1344.498718|
|      650.127.1634|
|      515.127.4561|
|011.44.1345.629268|
|      515.127.4562|
|011.44.1644.429264|
|011.44.1644.429262|
|      650.501.1876|
|      650.127.1834|
|011.44.1343.529268|
|011.44.1644.429265|
|      515.123.8181|
|      650.507.9833|
+------------------+
only showing top 20 rows



Есть ли сотрудники с одинаковыми фамилиями и сколько их.

In [ ]:
# код
sqlDF = spark.sql("""SELECT LAST_NAME, COUNT(LAST_NAME) as COUNTER
                     FROM employees
                     GROUP BY LAST_NAME
                     HAVING COUNT(LAST_NAME) > 1
""")
sqlDF.show()

+---------+-------+
|LAST_NAME|COUNTER|
+---------+-------+
|    Smith|      2|
|     King|      2|
|Cambrault|      2|
|   Taylor|      2|
+---------+-------+

